# 13c_encuestas Merge physical variables

## Import libraries

In [1]:
first_folder_path = '../../../'

In [2]:
import warnings
warnings.filterwarnings('ignore')
import geopandas as gpd
from geopandas.tools import overlay
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np
import seaborn as sns
import random
import scipy.stats as stats

import os
import sys
module_path = os.path.abspath(os.path.join(first_folder_path))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup
else:
    import aup

## Load physical variables and create unique edge_id

In [3]:
edges_var_gdl = gpd.read_file(first_folder_path + f"data/processed/volvo/edges_var_gdl/edges_var_gdl.gpkg")

# Show
print(edges_var_gdl.crs)
print(edges_var_gdl.shape)
edges_var_gdl.head(2)

EPSG:32613
(197439, 20)


,u,v,key,grade_abs,ndvi_mean,entropy,sidewalk_available,no_sidewalk_available,banqueta_pct,pobtot_u,pobtot_v,total_pobtot,average_population,dens_u,dens_v,average_density,interdens_u,interdens_v,average_intersdens,geometry
0,67637870229114485,67640019229114922,0,0.0,0.118923,1.144632,179.791185,69.176583,0.722146,80.0,27.0,107.0,53.5,153.566318,111.104363,132.335341,19.498764,19.071526,19.285145,"MULTILINESTRING ((676378.709 2291144.858, 6763..."
1,67640019229114922,67640526229117504,0,0.0,0.188798,0.317370,181.014513,68.547498,0.725329,27.0,5.0,32.0,16.0,111.104363,31.101049,71.102706,19.071526,19.041632,19.056579,"MULTILINESTRING ((676400.196 2291149.223, 6764..."


In [5]:
def create_unique_edge_id(edges, order='uvkey'):
    """
    Create a unique edge_id based on the 'u', 'v' and 'key' columns of the edges GeoDataFrame.

    Args:
        edges (geopandas.GeoDataFrame): GeoDataFrame containing the edges of the network.
        order (str, optional): Order for the unique id. Defaults to 'uvkey'.

    Returns:
        geopandas.GeoDataFrame: GeoDataFrame with the unique edge_id column.
    """
    # Turn ID data to string
    edges['u'] = edges['u'].astype('str')
    edges['v'] = edges['v'].astype('str')
    edges['key'] = edges['key'].astype('str')
    # Concatenate ID data to create unique edge_id
    if order == 'uvkey':
        edges['edge_id'] = edges['u']+edges['v']+edges['key']
    elif order == 'vukey':
        edges['edge_id'] = edges['v']+edges['u']+edges['key']
    # Turn ID data back to int
    edges['u'] = edges['u'].astype('int')
    edges['v'] = edges['v'].astype('int')
    edges['key'] = edges['key'].astype('int')

    return edges

In [6]:
edges_var_gdl = create_unique_edge_id(edges_var_gdl)

# Show
print(edges_var_gdl.shape)
edges_var_gdl.head(2)

(197439, 22)


,u,v,key,grade_abs,ndvi_mean,entropy,sidewalk_available,no_sidewalk_available,banqueta_pct,pobtot_u,...,average_population,dens_u,dens_v,average_density,interdens_u,interdens_v,average_intersdens,geometry,unique_edge_id,edge_id
0,67637870229114485,67640019229114922,0,0.0,0.118923,1.144632,179.791185,69.176583,0.722146,80.0,...,53.5,153.566318,111.104363,132.335341,19.498764,19.071526,19.285145,"MULTILINESTRING ((676378.709 2291144.858, 6763...",0 67637870229114485\n1 6764001...,67637870229114485676400192291149220
1,67640019229114922,67640526229117504,0,0.0,0.188798,0.317370,181.014513,68.547498,0.725329,27.0,...,16.0,111.104363,31.101049,71.102706,19.071526,19.041632,19.056579,"MULTILINESTRING ((676400.196 2291149.223, 6764...",0 67637870229114485\n1 6764001...,67640019229114922676405262291175040


In [7]:
edges_var_gdl.to_file(first_folder_path + f"data/processed/volvo/edges_var_gdl/edges_var_gdl_id.gpkg")

## Merge physical variables (edges) with edges that have route codes (Manually selected following coincidence with manually drawn routes)

### __Load data__ - Physical variables

### __Load data__ - Routes with codes

In [5]:
rutas_edges = gpd.read_file(first_folder_path + f"data/processed/volvo/encuestas/03_rutas_en_edges/rutas_edges.gpkg")

# Show
print(rutas_edges.crs)
print(rutas_edges.shape)
rutas_edges.head(2)

EPSG:4326
(1865, 2)


,code,geometry
0,JC1V008,"LINESTRING (-103.38511 20.68946, -103.38498 20..."
1,JC1V008,"LINESTRING (-103.38498 20.69015, -103.38489 20..."


### Merge using geometry (Failed, using Hausdorff)

In [46]:
rutas_edges_32613 = rutas_edges.copy()
rutas_edges_32613.to_crs("EPSG:32613",inplace=True)

# Show
print(rutas_edges_32613.crs)
print(rutas_edges_32613.shape)
rutas_edges_32613.head(2)

EPSG:32613
(1865, 2)


,code,geometry
0,JC1V008,"LINESTRING (668194.953 2288617.514, 668207.517..."
1,JC1V008,"LINESTRING (668207.517 2288693.987, 668216.476..."


In [48]:
edges_var_gdl.geom_type.unique()

array(['MultiLineString'], dtype=object)

In [49]:
rutas_edges_32613.geom_type.unique()

array(['LineString'], dtype=object)

In [26]:
# Convert LineStrings to MultiLineStrings since edges_var_gdl has MultiLineStrings only
from shapely.geometry import MultiLineString, LineString

def to_multi(geom):
    if isinstance(geom, LineString):
        return MultiLineString([geom])
    return geom  # ya es MultiLineString

rutas_edges_32613['geometry'] = rutas_edges_32613['geometry'].apply(to_multi)

In [44]:
rutas_vars = rutas_edges_32613.merge(edges_var_gdl,
                                     on='geometry',
                                     how='left',
                                     suffixes=('', '_orig')
                                    )

# Show
print(rutas_vars.crs)
print(rutas_vars.shape)
rutas_vars.head(2)

EPSG:32613
(1865, 21)


,code,geometry,u,v,key,grade_abs,ndvi_mean,entropy,sidewalk_available,no_sidewalk_available,...,pobtot_u,pobtot_v,total_pobtot,average_population,dens_u,dens_v,average_density,interdens_u,interdens_v,average_intersdens
0,JC1V008,"MULTILINESTRING ((668194.953 2288617.514, 6682...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,JC1V008,"MULTILINESTRING ((668207.517 2288693.987, 6682...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
rutas_vars.loc[rutas_vars.code=='JC1V008']

,code,geometry,u,v,key,grade_abs,ndvi_mean,entropy,sidewalk_available,no_sidewalk_available,...,pobtot_u,pobtot_v,total_pobtot,average_population,dens_u,dens_v,average_density,interdens_u,interdens_v,average_intersdens
0,JC1V008,"MULTILINESTRING ((668194.953 2288617.514, 6682...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,JC1V008,"MULTILINESTRING ((668207.517 2288693.987, 6682...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,JC1V008,"MULTILINESTRING ((668216.476 2288748.459, 6682...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JC1V008,"MULTILINESTRING ((668189.03 2288541.706, 66819...",6.681890e+16,6.681950e+16,0.0,0.013094,0.109648,1.530083,326.412829,0.0,...,88.0,102.0,190.0,95.0,59.608287,65.548800,62.578543,13.723045,12.761922,13.242484
4,JC1V008,"MULTILINESTRING ((667635.629 2288481.5, 667807...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,JC1V008,"MULTILINESTRING ((667807.619 2288478.06, 66799...",6.678076e+16,6.679934e+16,0.0,0.010778,0.160844,0.814094,534.569406,0.0,...,49.0,28.0,77.0,38.5,38.368577,39.212383,38.790480,17.597582,16.147524,16.872553
6,JC1V008,"MULTILINESTRING ((667613.6 2288481.868, 667635...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,JC1V008,"MULTILINESTRING ((667349.293 2288486.288, 6676...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,JC1V008,"MULTILINESTRING ((668183.884 2288470.566, 6681...",6.681839e+16,6.681890e+16,0.0,0.013959,0.116048,0.971241,319.983761,0.0,...,86.0,88.0,174.0,87.0,62.599545,59.608287,61.103916,14.745478,13.723045,14.234262
9,JC1V008,"MULTILINESTRING ((668005.265 2288474.124, 6681...",6.680053e+16,6.681839e+16,0.0,0.016817,0.148290,0.457853,508.901920,0.0,...,50.0,86.0,136.0,68.0,74.236681,62.599545,68.418113,16.091099,14.745478,15.418288


In [42]:
rutas_edges_32613.loc[rutas_edges_32613.code=='JC1V008']

,code,geometry
0,JC1V008,"MULTILINESTRING ((668194.953 2288617.514, 6682..."
1,JC1V008,"MULTILINESTRING ((668207.517 2288693.987, 6682..."
2,JC1V008,"MULTILINESTRING ((668216.476 2288748.459, 6682..."
3,JC1V008,"MULTILINESTRING ((668189.03 2288541.706, 66819..."
4,JC1V008,"MULTILINESTRING ((667635.629 2288481.5, 667807..."
5,JC1V008,"MULTILINESTRING ((667807.619 2288478.06, 66799..."
6,JC1V008,"MULTILINESTRING ((667613.6 2288481.868, 667635..."
7,JC1V008,"MULTILINESTRING ((667349.293 2288486.288, 6676..."
8,JC1V008,"MULTILINESTRING ((668183.884 2288470.566, 6681..."
9,JC1V008,"MULTILINESTRING ((668005.265 2288474.124, 6681..."


In [36]:
rutas_edges_32613.loc[~rutas_edges_32613.code.isin(rutas_vars.code.unique())]

,code,geometry
87,MeV1V003,"MULTILINESTRING ((668242.076 2288898.785, 6682..."
130,DP1M001,"MULTILINESTRING ((668445.733 2288864.148, 6682..."
131,DP1M001,"MULTILINESTRING ((668242.076 2288898.785, 6682..."
132,DP1M001,"MULTILINESTRING ((668257.265 2288959.632, 6682..."
162,DP1V029,"MULTILINESTRING ((668643.365 2289663.112, 6686..."
163,DP1V029,"MULTILINESTRING ((668833.646 2289652.659, 6688..."
164,DP1V029,"MULTILINESTRING ((668825.188 2289730.649, 6688..."
165,DP1V029,"MULTILINESTRING ((668555.253 2289754.843, 6685..."
171,DP1V031,"MULTILINESTRING ((668612.929 2289784.714, 6685..."
172,DP1V031,"MULTILINESTRING ((668793.073 2289877.222, 6686..."


### __Load data__ - final routes (File has polls data)

In [3]:
final_routes = gpd.read_file(first_folder_path + f"data/processed/volvo/encuestas/02_rutas_dibujadas/final_routes.gpkg")

# Show
print(final_routes.crs)
print(final_routes.shape)
final_routes.head(2)

EPSG:32613
(201, 70)


,Código,layer,path,field_1,field_2,field_3,field_4,field_5,field_6,field_7,...,field_59,field_60,field_61,field_62,field_63,field_64,field_65,field_66,field_67,geometry
0,DP3V002,Rutas Danaé,C:/Users/sarah/Desktop/Levantamiento VREF/UNI...,252,9/19/24 18:40:07,9/19/24 18:46:58,anonymous,None,None,DP,...,De acuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",Totalmente de acuerdo,Totalmente de acuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,"LINESTRING (660511.087 2283608.716, 660443.453..."
1,DP3V005,Rutas Danaé,C:/Users/sarah/Desktop/Levantamiento VREF/UNI...,255,9/19/24 18:50:40,9/19/24 18:57:14,anonymous,None,None,DP,...,De acuerdo,De acuerdo,De acuerdo,De acuerdo,De acuerdo,De acuerdo,En desacuerdo,De acuerdo,De acuerdo,"LINESTRING (660562.586 2283275.252, 660591.611..."
